# PROCESS MINING НА ТЕКСТОВЫХ ЧАТАХ ПО СДЕЛКАМ С НЕДВИЖИМОСТЬЮ

Для совершения сделок с недвижимостью обычно требуется предоставить пакет документов, который зависит от различных факторов. При уточнении списка документов клиенты активно используют текстовые чаты. От эффективности взаимодействия консультантов с клиентами, зависит успешность сделок. Одним из критериев оценки эффективности является минимизация времени, которое тратит клиент на подготовку документов.  Взаимодействие консультантов с клиентами можно представить в виде процесса, в рамках которого последовательно запрашиваются документы. Таких процессов достаточно много, и они не формализованы. Поэтому было необходимо провести анализ чатов, выделить наиболее часто встречаемые процессы и т. д.

У каждого чата есть свой уникальный идентификатор, у сообщения – тег отправителя, дата и время создания.

In [ ]:
![alt text](image1.png "Title")

In [ ]:
![caption](files/image1.png)